<a href="https://colab.research.google.com/github/mathu3004/Pearl_Path/blob/Personalized_Itinerary_Generator_Based_Radius/Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the datasets
hotels_df = '/content/drive/My Drive/DataPre/colomboHotels_processed.csv'
hotels_df = pd.read_csv(hotels_df)
restaurants_df = '/content/drive/My Drive/DataPre/colomboRestaurants_processed.csv'
restaurants_df = pd.read_csv(restaurants_df)
attractions_df = '/content/drive/My Drive/DataPre/merged_processed_data_Attractions.csv'
attractions_df = pd.read_csv(attractions_df)
user_inputs_df = '/content/drive/My Drive/DataPre/preprocessed_user_inputs.csv'
user_inputs_df = pd.read_csv(user_inputs_df)

# Display the first few rows of each dataset
hotels_df.head(), restaurants_df.head(), attractions_df.head(), user_inputs_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(                                             address  \
 0  14 Borella Cross Road off Ward Place, Colombo ...   
 1  538 Galle Road Colombo 03, Colombo 00300 Sri L...   
 2        No 7/1 Elias Place, Colombo 01000 Sri Lanka   
 3  29 Milagiriya Avenue Marine Drive, Colombo 004...   
 4        28, Borella Cross Road, Colombo 8 Sri Lanka   
 
                                        all_amenities category  \
 0  Internet, Free Internet, Restaurant, Airport t...    hotel   
 1  Internet, Room service, Free Internet, Wheelch...    hotel   
 2  Internet, Free Internet, Free parking, Wifi, F...    hotel   
 3  Wifi, Public Wifi, Free Wifi, Dry Cleaning, Bu...    hotel   
 4  Internet, Free Internet, Free parking, Air con...    hotel   
 
   categoryReviewScores_0_categoryName categoryReviewScores_1_categoryName  \
 0                            Location                               Rooms   
 1                            Location                               Rooms   
 2                      

In [15]:
!pip install ace-tools

In [16]:
# Function to get top 5 hotels based on rating and price
def get_top_hotels(hotels_df, place, top_n=5):
    # Filter hotels by place
    filtered_hotels = hotels_df[hotels_df['address'].str.contains(place, case=False, na=False)]

    # Sort hotels by rating (descending) and price (ascending)
    top_hotels = filtered_hotels.sort_values(by=['rating', 'priceRange_LKR'], ascending=[False, True]).head(top_n)

    return top_hotels[['name', 'address', 'rating', 'priceRange_LKR', 'latitude', 'longitude', 'webUrl']]

# Get top 5 hotels in Colombo
top_hotels = get_top_hotels(hotels_df, "Colombo")

display(top_hotels)

,name,address,rating,priceRange_LKR,latitude,longitude,webUrl
44,Mini's Residence,"136/3 Polhengoda Road, Colombo 00500 Sri Lanka",5.0,7426.0,6.927079,79.861244,https://www.tripadvisor.com/Hotel_Review-g2939...
76,Colombo Riverdale Hotel,"147/2, Off Peliyagoda Bridge Sedawatta - Ambat...",5.0,7862.5,6.953510,79.883320,https://www.tripadvisor.com/Hotel_Review-g2939...
87,Star Class Comfort Capital City,"648, By Road 1, Off Subuthi Mawatha, Sri Subut...",5.0,8736.0,6.907302,79.916855,https://www.tripadvisor.com/Hotel_Review-g2939...
99,My Little Island Hostel,"3 594/1, Galle Road, Colombo #594, Colombo 003...",5.0,8736.0,6.899937,79.853110,https://www.tripadvisor.com/Hotel_Review-g2939...
213,Vista Villa Raha,"No 33 Macleod Road, Colombo 00400 Sri Lanka",5.0,10192.0,6.884857,79.860930,https://www.tripadvisor.com/Hotel_Review-g2939...


In [17]:
# Function to ask user to select a hotel
def select_hotel(top_hotels):
    print("Please select a hotel from the following options:")
    for i, hotel in enumerate(top_hotels['name'], start=1):
        print(f"{i}. {hotel}")

    # Get user selection
    selection = int(input("Enter the number of your selected hotel: "))

    # Retrieve selected hotel details
    selected_hotel = top_hotels.iloc[selection - 1]

    return selected_hotel

# Call the function to let user select a hotel
selected_hotel = select_hotel(top_hotels)

# Display the selected hotel
selected_hotel

Please select a hotel from the following options:
1. Mini's Residence
2. Colombo Riverdale Hotel
3. Star Class Comfort Capital City
4. My Little Island Hostel
5. Vista Villa Raha
Enter the number of your selected hotel: 4


,99
name,My Little Island Hostel
address,"3 594/1, Galle Road, Colombo #594, Colombo 003..."
rating,5.0
priceRange_LKR,8736.0
latitude,6.899937
longitude,79.85311
webUrl,https://www.tripadvisor.com/Hotel_Review-g2939...


In [18]:
# Get maximum range from user
max_distance = float(input("Enter the maximum range from the hotel (in km): "))

Enter the maximum range from the hotel (in km): 35


In [39]:
import numpy as np

# Haversine formula to calculate the great-circle distance between two coordinates
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in km
    try:
        lat1, lon1, lat2, lon2 = map(np.radians, [float(lat1), float(lon1), float(lat2), float(lon2)]) # Convert to float before calling np.radians
    except ValueError:
        return np.nan  # Return NaN if conversion fails
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c  # Distance in km

# Function to filter attractions and restaurants within user-specified range
def filter_places_within_range(selected_hotel, attractions_df, restaurants_df, max_distance):
    hotel_lat, hotel_lon = selected_hotel['latitude'], selected_hotel['longitude']

    # Calculate distances and filter attractions
    attractions_df['distance_from_hotel'] = attractions_df.apply(
        lambda row: haversine(hotel_lat, hotel_lon, row['Latitude'], row['Longitude']), axis=1)
    filtered_attractions = attractions_df[attractions_df['distance_from_hotel'] <= max_distance]

    # Calculate distances and filter restaurants
    restaurants_df['distance_from_hotel'] = restaurants_df.apply(
        lambda row: haversine(hotel_lat, hotel_lon, row['latitude'], row['longitude']), axis=1)
    filtered_restaurants = restaurants_df[restaurants_df['distance_from_hotel'] <= max_distance]

    return filtered_attractions, filtered_restaurants

# Function to allocate daily restaurants and attractions
def create_daily_itinerary(filtered_attractions, filtered_restaurants, num_days):
    daily_plan = []

    for day in range(num_days):
        daily_meals = {}

        # Select 3 restaurants per meal (breakfast, lunch, dinner)
        for meal in ['Breakfast', 'Lunch', 'Dinner']:
            meal_restaurants = filtered_restaurants[filtered_restaurants['mealTypes'].str.contains(meal, case=False, na=False)]
            daily_meals[meal] = meal_restaurants.sample(n=min(3, len(meal_restaurants)), replace=False).to_dict('records')

        # Select 3-4 attractions per day, ensuring no repetition
        daily_attractions = filtered_attractions.sample(n=min(4, len(filtered_attractions)), replace=False).to_dict('records')

        daily_plan.append({
            "day": day + 1,
            "restaurants": daily_meals,
            "attractions": daily_attractions
        })

    return daily_plan

# Function to sort places based on rating, price, and ranking
def rank_places(places_df):
    return places_df.sort_values(by=['Rating', 'Lowest Price', 'Ranking_Position'], ascending=[False, True, True])

# Function to sort places based on rating, price, and ranking
def rank_rest(places_df):
    return places_df.sort_values(by=['rating', 'priceLevel', 'rankingPosition'], ascending=[False, True, True])

# Function to integrate collaborative filtering (future scope)
def collaborative_filtering(user_history, recommendations):
    # Placeholder for collaborative filtering logic (can be implemented with ML models)
    return recommendations

# Function to integrate Google Maps API for real-time distances (future scope)
def get_real_time_distances(selected_hotel, attractions, restaurants):
    # Placeholder function for Google Maps API integration
    return attractions, restaurants

# The complete flow after the user selects a hotel
def generate_itinerary(selected_hotel, max_distance, num_days):
    # Step 1: Filter places within range
    filtered_attractions, filtered_restaurants = filter_places_within_range(selected_hotel, attractions_df, restaurants_df, max_distance)

    # Step 2: Rank places
    ranked_attractions = rank_places(filtered_attractions)
    ranked_restaurants = rank_rest(filtered_restaurants)

    # Step 3: Create a daily itinerary
    itinerary = create_daily_itinerary(ranked_attractions, ranked_restaurants, num_days)

    return itinerary


itinerary = generate_itinerary(selected_hotel, max_distance, num_days=3)
# Function to print the itinerary with names only
def print_itinerary(itinerary):
    for day in itinerary:
        print(f"Day {day['day']}:")
        print("  Attractions:", ", ".join([attraction["Name"] for attraction in day["attractions"]]))
        print("  Restaurants:")
        for meal, places in day["restaurants"].items():
            print(f"    {meal}: {', '.join([restaurant['name'] for restaurant in places])}")
        print("\n")

# Print the itinerary
print_itinerary(itinerary)



Day 1:
  Attractions: Colombo City Walks, Vacation Lanka Tours, Travel Sri Lankan Tours, Raja Jewellers
  Restaurants:
    Breakfast: Shanmugas, Sri Vihar, Beira Kitchen
    Lunch: Shoulders by Harpos, Silk Route, AYU
    Dinner: Barefoot Garden Cafe, SSC Singhalease Sports Club, Chola Authentic Indian Restaurant


Day 2:
  Attractions: Wolvendaal Church, Amara Lanka Tours & Travels, Tour Driver in Sri Lanka, Urban Island
  Restaurants:
    Breakfast: Harbour Court, The Kingsbury, The ‘Stache, The Angler
    Lunch: Saraswathie Lodge, Yarl Hotel, Sapphyr Lounge, Shangri-La Colombo
    Dinner: Cafe Francais, LOON TAO, Tasty Caterers


Day 3:
  Attractions: Cooking Class Colombo, Barefoot, Nature's Secrets Beauty Spa & Boutique, South Lanka Tours - Day Tours
  Restaurants:
    Breakfast: The t-Lounge by Dilmah, Lago Tropical Italian Bistro, Colombo Swimming Club
    Lunch: The Four Leafed Clover, The t-Lounge by Dilmah, Graze Kitchen
    Dinner: Tsing Tao, Chola Authentic Indian Restauran